In [1]:
# Allow multiple outputs in a single cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Analysis of SleepCycle(TM) data

Let's start by downloading the data from SleepSecure(TM). This might take a while the first time, as the data is downloaded from a *relatively* slow server:

In [2]:
from quelf.sleepcycle import SleepCycle

sc = SleepCycle()
data = sc.data

How large is our dataset?

In [3]:
len(data)

820

What does our data consist of?

In [4]:
data.columns

Index(['events', 'start', 'stop'], dtype='object')

What does our latest data point look like?

In [5]:
data.loc[0]

events    [[60.0, 3, 0.11230000000000001], [60.0, 1, 0.0...
start                                   2018-03-31 01:09:58
stop                                    2018-03-31 09:03:21
Name: 0, dtype: object

## Data processing
Let's calculate how long each sleep recording lasts:

In [6]:
time_slept = data['stop'] - data['start']
data['time_slept'] = time_slept
time_slept.describe()

count                       820
mean     0 days 08:48:10.129268
std      0 days 01:22:09.785122
min             0 days 01:09:58
25%      0 days 08:06:46.750000
50%      0 days 08:36:09.500000
75%      0 days 09:25:50.750000
max             0 days 14:20:10
dtype: object

#### Outlier detection
The dataset might contain some outliers. If these maximum and/or minimum values look incorrect to you, you might want to delete the datapoints from the SleepCycle app, delete the contents of the ``data`` directory, and rerun the notebook.

In [7]:
# Show the maximum and minimum time slept, and on which date they occur
data[data['time_slept'] == data['time_slept'].max()]
data[data['time_slept'] == data['time_slept'].min()]

,events,start,stop,time_slept
178,"[[60.0, 3, 0.3593], [60.0, 1, 0.53549999999999...",2017-09-30 01:42:30,2017-09-30 16:02:40,14:20:10


,events,start,stop,time_slept
86,"[[60.0, 1, 0.062399999999999005], [60.0, 3, 0....",2018-01-07 03:14:30,2018-01-07 04:24:28,01:09:58


Set ``MAX_HOURS`` and ``MIN_HOURS`` to a value which could indicate something being wrongly recorded into SleepCycle.

In [8]:
MAX_HOURS = 13
MIN_HOURS = 4

data[data['time_slept'].dt.total_seconds() > MAX_HOURS*60*60]
data[data['time_slept'].dt.total_seconds() < MIN_HOURS*60*60]

,events,start,stop,time_slept
178,"[[60.0, 3, 0.3593], [60.0, 1, 0.53549999999999...",2017-09-30 01:42:30,2017-09-30 16:02:40,14:20:10
291,"[[60.0, 1, 0.0454], [60.0, 3, 0.0086], [120.0,...",2017-06-03 00:22:45,2017-06-03 13:32:00,13:09:15
353,"[[60.0, 1, 0.047399999999999005], [60.0, 3, 0....",2017-04-01 01:11:42,2017-04-01 14:44:52,13:33:10
442,"[[60.0, 3, 0.0018000000000000002], [60.0, 1, 0...",2017-01-03 01:25:44,2017-01-03 14:30:02,13:04:18
571,"[[60.0, 3, 0.010400000000000001], [60.0, 1, 0....",2016-08-26 00:50:44,2016-08-26 15:03:34,14:12:50
788,"[[60.0, 1, 0.06419999999999901], [120.0, 1, 0....",2016-01-14 21:43:36,2016-01-15 11:27:13,13:43:37


,events,start,stop,time_slept
66,"[[60.0, 1, 0.057799999999999005], [60.0, 3, 0....",2018-01-25 01:13:29,2018-01-25 04:13:31,03:00:02
67,"[[60.0, 3, 0.003899999999999], [60.0, 1, 0.054...",2018-01-24 22:49:56,2018-01-25 01:06:58,02:17:02
86,"[[60.0, 1, 0.062399999999999005], [60.0, 3, 0....",2018-01-07 03:14:30,2018-01-07 04:24:28,01:09:58
339,"[[60.0, 1, 0.068], [60.0, 3, 0.0178], [120.0, ...",2017-04-15 01:03:35,2017-04-15 02:22:44,01:19:09
372,"[[60.0, 1, 0.053199999999999005], [60.0, 3, 0....",2017-03-13 00:02:41,2017-03-13 02:51:53,02:49:12
410,"[[60.0, 3, 0.023699999999999003], [60.0, 1, 0....",2017-02-03 22:58:28,2017-02-04 01:37:04,02:38:36


In [9]:
from datetime import timedelta
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

# Fill the width of a 1920x1080 screen when plotting the entire dataset
plt.rcParams['figure.figsize'] = [25, 6]

# Use FiveThirtyEight matplotlib theme
plt.style.use('fivethirtyeight')

# Calculate time went to bed, as a offset relative to 18:00
went_to_bed = (data['start'].dt.hour + (data['start'].dt.second / 60) - 18) % 24

# Decimal hours slept
hours_slept = time_slept.dt.total_seconds() / (60 * 60)

# Create colormap relative to hours slept
norm = (hours_slept - hours_slept.min()) / hours_slept.max()
cmap = cm.get_cmap()
color_map = cmap(norm)

# Bottom of barplot relative to 18:00, height relative to hours slept
ax = plt.bar(data['start'].values, hours_slept, bottom=went_to_bed, width=1.125, color=color_map)

xlabel = plt.xlabel('Date')
ylabel = plt.ylabel('Time in bed')

# Map ticks relative to 18:00
yticks = plt.yticks((np.arange(2, 25, 2) - 18) % 24, (np.arange(2, 25, 2)))

plt.show()

<Figure size 2500x600 with 1 Axes>

## SleepSecure(TM) API exploration
SleepSecure offers some API endpoints, which we will now explore:

In [12]:
sc.fetch(endpoint='/stat/session', params={'id': '5718148320854016', 'next': '1'})

{'alarm_mode': 0,
 'graph': [[0, 93],
  [1, 90.81128765555019],
  [2, 93.81734869886975],
  [3, 91.41718808493046],
  [4, 90.92837130719435],
  [5, 92.5407550380968],
  [6, 91.60734079106012],
  [7, 91.12688499322337],
  [8, 95.31574993906199],
  [9, 94.09838048195674],
  [10, 93.70957160890873],
  [11, 81.80254129400353],
  [12, 68.4896960268396],
  [13, 45.586008796386714],
  [14, 31.10301542173655],
  [15, 28.10611945666922],
  [16, 34.98543473349164],
  [17, 46.451946720300725],
  [18, 59.693460356926956],
  [19, 70.55689564047569],
  [20, 74.7009226241703],
  [21, 71.08870718979641],
  [22, 60.45222278856013],
  [23, 46.40947719680005],
  [24, 31.402129813542178],
  [25, 21.045342248048023],
  [26, 14.70985225360291],
  [27, 14.334175584309397],
  [28, 22.97708325404308],
  [29, 30.69983323389047],
  [30, 39.8990105574951],
  [31, 37.41060509691733],
  [32, 27.678212537162523],
  [33, 25.161529056154894],
  [34, 22.007572820509615],
  [35, 27.596682389318346],
  [36, 38.3688557158